In [1]:
import csv
import requests
from bs4 import BeautifulSoup
from datetime import date
import os
import re
import json
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from os import listdir
from os.path import isfile, join

In [2]:
# Make file paths
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(os.path.abspath(''))))
json_data_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/json_data')
csv_data_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/csv')
images_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/images')
svg_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/images/svg')
png_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/images/png')
gif_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/images/gif/game-modified3.gif')

path_list = [images_path, svg_path, png_path, gif_path, json_data_path, csv_data_path]
csv_data_path

'/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/csv'

In [3]:
def game_url_builder(game_name: str, game_code: str, current_year: int, current_season: str):
    backstabbr_url = 'https://www.backstabbr.com/game/'
    first_year = 1901
    
    base_url = backstabbr_url + game_name + '/' + game_code + '/'
    seasons = ['spring', 'fall', 'winter']
    season_urls = [str(base_url+'****/spring'), str(base_url+'****/fall'), str(base_url+'****/winter')]
    first_url = season_urls[0].replace("****", str(first_year)) 
    
    url_list = []
    years_list = []
    season_list = []
    
    # Calculate number of urls to build
    number_of_years = current_year - first_year + 1
    base_number = number_of_years*3
    if current_season=="spring":
        number_of_urls = base_number + 1
    elif current_season=="fall":
        number_of_urls = base_number + 2
    else:
        number_of_urls = base_number + 3
    if number_of_years==0:
        years_list.append('1901')
    else:
        for num in range(0,number_of_years):
            year = first_year+num
            years_list.append(str(year))
    current_url_number = 0
    for i, current_year in enumerate(years_list):
        if current_url_number == number_of_urls:
            break
        year_url_list = []
        for j, item in enumerate(season_urls):
            season_list.append(seasons[j])
            if current_url_number == number_of_urls:
                break
            year_url_list.append(item.replace("****", str(current_year)))
            current_url_number+=1
        for url in year_url_list:
            url_list.append(url) 
#     url_list.append(base_url)
    return url_list, years_list, season_list  

In [4]:
# War on drugs game
game_name = 'The-war-on-drugs'
game_code = '5704487623196672'

In [5]:
urls, years, seasons = game_url_builder(game_name, game_code, 1921, 'winter')

In [6]:
ref_data_tuples = list(zip(urls, years, seasons))
game_reference_data = pd.DataFrame(ref_data_tuples, columns=['url', 'year', 'season'])

In [7]:
game_reference_data.to_csv(csv_data_path + '/reference_data/game_reference_data.csv')
game_reference_data.head()

,url,year,season
0,https://www.backstabbr.com/game/The-war-on-dru...,1901,spring
1,https://www.backstabbr.com/game/The-war-on-dru...,1902,fall
2,https://www.backstabbr.com/game/The-war-on-dru...,1903,winter
3,https://www.backstabbr.com/game/The-war-on-dru...,1904,spring
4,https://www.backstabbr.com/game/The-war-on-dru...,1905,fall


In [8]:
len(years)*3
len(seasons)
len(urls)

63

In [9]:
urls[0][66:70]

'1901'

In [64]:
# Get all orders data for the game
def get_script_data(script_string, beginning, end):
        start = script_string.find(beginning) + len(beginning)
        end = script_string.find(end)
        substring = script_string[start:end]
        return str(substring)

for i, url in enumerate(urls):
    # Get game page here
    page = requests.get(url).content
    soup = BeautifulSoup(page)
    # Find all script tags in the page
    all_scripts = soup.find_all('script')
    # Desired order json is the 10th script. Order does not change, unless there are changes to the Backstabbr code!
    javascript_script_target = all_scripts[10]
    javascript_string = str(javascript_script_target)
    orders_string = get_script_data(javascript_string, "var orders = ", ";\n  var unitsByPlayer")
    orders_json = json.loads(orders_string)
    unitsByPlayer_string = get_script_data(javascript_string, 'var unitsByPlayer = ', ";\n  var territories =")
    unitsByPlayer_json = json.loads(unitsByPlayer_string)
    territories_string = get_script_data(javascript_string, 'var territories = ', ";\n  var mapSize")
    territories_json = json.loads(territories_string)
    stage_string = get_script_data(javascript_string, 'var stage = ', ';\n  var orders')
    stage_json = json.loads(stage_string)
    unitChangeCount_string = get_script_data(javascript_string, 'var unitChangeCount = ', ';\n  var buildableTerritories =')
    unitChangeCount_json = json.loads(unitChangeCount_string)
    json_list = [orders_json, unitsByPlayer_json, territories_json, stage_json, unitChangeCount_json]
    json_filename = ['orders', 'unitsByPlayer', 'territories', 'stage', 'unitChangeCount']
    for i, item in enumerate(json_list):
        with open(json_data_path + '/' + url[66:70] + '_' + url[71:] + '_' + str(json_filename[i]) + '_' + '.txt', 'w') as outfile:
            json.dump(item, outfile)
    sleep(randint(3,10))

IndexError: list index out of range

In [13]:
# Data preparation functions
# Wrangle units data into dataframe
def getUnitsByPlayer(unitsByPlayerData, year, season, season_number):
    country_list = []
    territory_list = []
    unit_type_list = []
    years_list = []
    seasons_list = []
    season_number_list = []
    for i, player in enumerate(unitsByPlayerData.items()):
        units = player[1]
        for territory, unit_type in units.items():
            country_list.append(player[0])
            territory_list.append(territory)
#             print(unit_type)
            if unit_type=={'type': 'F', 'coast': 'sc'}:
                unit_type_list.append('F')
            else:
                unit_type_list.append(unit_type)
            years_list.append(year)
            seasons_list.append(season)
            season_number_list.append(season_number)
#             print(year, season, unit_type)
    data_tuples = list(zip(country_list, territory_list, unit_type_list, years_list, seasons_list, season_number_list))
    df = pd.DataFrame(data_tuples, columns=['country','territory', 'unit_type', 'year', 'season', 'season_number'])
    return df

# Wrangle UnitChangeCounts into dataframe
def getUnitChangeCounts(unitChangeCountData, year, season_number, season='winter'):
    country_list = []
    unitChangeCount_list = []
    years_list = []
    seasons_list = []
    season_number_list = []
    for player in unitChangeCountData.items():
        country_list.append(player[0])
        unitChangeCount_list.append(player[1])
        years_list.append(year)
        seasons_list.append(season)
        season_number_list.append(season_number)
    data_tuples = list(zip(country_list, unitChangeCount_list, years_list, seasons_list, season_number_list))
    df = pd.DataFrame(data_tuples, columns=['country','unitChange', 'year', 'season', 'season_number'])
    return df

# Wrangle orders data into dataframe
def getOrders(orderData, year, season, season_number):
    country_list = []
    type_list = []
    unitFrom_list = []
    unitTo_list = []
    unitType_list = []
    result_list = []
    result_reason_list = []
    years_list = []
    seasons_list = []
    season_number_list = []
    num_orders = 0
#     WINTER unit_type": "F", "type": "BUILD", "result": "SUCCEEDS"},
    for player in orderData.items():
        for territory in player[1].items():
            if 'unit_type' in territory[1]:
                unitTo_list.append(np.nan)
                unitType_list.append(territory[1]['unit_type'])
                type_list.append(territory[1]['type'])
                result_list.append(territory[1]['result'])
                result_reason_list.append(np.nan)
                
            elif 'to' in territory[1]:
                unitTo_list.append(territory[1]['to'])
                type_list.append(territory[1]['type'])
                result_list.append(territory[1]['result'])
                result_reason_list.append(territory[1]['result_reason'])
                unitType_list.append(np.nan)
            else:
                if 'result_reason' not in territory[1]:
                    unitTo_list.append(np.nan)
                    type_list.append(territory[1]['type'])
                    result_list.append(territory[1]['result'])
                    result_reason_list.append(np.nan)
                    unitType_list.append(np.nan)
                else:
                    unitTo_list.append(np.nan)
                    type_list.append(territory[1]['type'])
                    result_list.append(territory[1]['result'])
                    result_reason_list.append(territory[1]['result_reason'])
                    unitType_list.append(np.nan)
            num_orders+=1
            country_list.append(player[0])
            unitFrom_list.append(territory[0])                
            years_list.append(year)
            seasons_list.append(season)
            season_number_list.append(season_number)
    print("Test1 = numbers should all be equal: ",len(country_list), len(type_list), len(unitFrom_list), len(unitTo_list), len(result_list),len(result_reason_list), len(years_list))
    data_tuples = list(zip(country_list, type_list, unitFrom_list, unitTo_list, result_list, result_reason_list, years_list, seasons_list, season_number_list, unitType_list))
    df = pd.DataFrame(data_tuples, columns=['country', 'type', 'unitFrom','unitTo','result', 'result_reason','year', 'season', 'season_number', 'unitType'])    
    print("Test2 = this number should be equal to the above set of numbers: ", num_orders)

    return df

In [14]:
filetypes = ['orders', 'stage', 'territories', 'unitChangeCount', 'unitsByPlayer']
year_index = -1
orders_data_combined = []
UnitsByPlayer_data_combined = []
unitChangeCount_data_combined = []
for file_type in filetypes:
    with open(json_data_path + '/1901_spring_'+str(file_type)+'_.txt') as json_file:
        data = json.load(json_file)
    if file_type=='orders':
        orders_data_combined = getOrders(data, '1901', 'spring', 1)
    elif file_type=='unitChangeCount':
        unitChangeCount_data_combined = getUnitChangeCounts(data, '1901', 1, 'spring')
    elif file_type=='unitsByPlayer':
        UnitsByPlayer_data_combined = getUnitsByPlayer(data, '1901', 'spring', 1)
    else:
        continue
year_index=0
for i, season in enumerate(seasons[1:]):
    season_number = i+2   
    print(season, years[year_index])
    for file_type in filetypes:
        with open(json_data_path + '/' + years[year_index] +'_'+str(season)+'_'+str(file_type)+'_.txt') as json_file:
            data = json.load(json_file)
        if file_type == 'orders':
            orders_data = getOrders(data, str(years[year_index]), season, season_number)
            orders_data_combined = orders_data_combined.append(orders_data, ignore_index = True)
        elif file_type == 'unitsByPlayer':
            UnitsByPlayer_data = getUnitsByPlayer(data, str(years[year_index]), season, season_number)
            UnitsByPlayer_data_combined = UnitsByPlayer_data_combined.append(UnitsByPlayer_data, ignore_index = True)
        elif file_type == 'unitChangeCount':
            unitChangeCount_data = getUnitChangeCounts(data, str(years[year_index]), season_number, season)
            unitChangeCount_data_combined = unitChangeCount_data_combined.append(unitChangeCount_data, ignore_index = True)
        else:
            continue
    if season_number%3==0:
        year_index+=1
full_data_list = [orders_data_combined, UnitsByPlayer_data_combined, unitChangeCount_data_combined]
full_datanames_list = ['orders', 'unitsByPlayer', 'unitChangeCount']

# WHY IS THIS HAVING AN ERROR WITH result_reason??

Test1 = numbers should all be equal:  22 22 22 22 22 22 22
Test2 = this number should be equal to the above set of numbers:  22
fall 1901
Test1 = numbers should all be equal:  22 22 22 22 22 22 22
Test2 = this number should be equal to the above set of numbers:  22
winter 1901
Test1 = numbers should all be equal:  11 11 11 11 11 11 11
Test2 = this number should be equal to the above set of numbers:  11
spring 1902
Test1 = numbers should all be equal:  33 33 33 33 33 33 33
Test2 = this number should be equal to the above set of numbers:  33
fall 1902
Test1 = numbers should all be equal:  32 32 32 32 32 32 32
Test2 = this number should be equal to the above set of numbers:  32
winter 1902
Test1 = numbers should all be equal:  3 3 3 3 3 3 3
Test2 = this number should be equal to the above set of numbers:  3
spring 1903
Test1 = numbers should all be equal:  34 34 34 34 34 34 34
Test2 = this number should be equal to the above set of numbers:  34
fall 1903
Test1 = numbers should all be equa

In [15]:
rslt_df = orders_data_combined.loc[orders_data_combined['year'] == "1902"]

In [16]:
rslt_df

,country,type,unitFrom,unitTo,result,result_reason,year,season,season_number,unitType
55,Turkey,HOLD,Ank,NaN,SUCCEEDS,Unchallenged,1902,spring,4,NaN
56,Turkey,MOVE,Bul,Rum,SUCCEEDS,Attack strength is greater,1902,spring,4,NaN
57,Turkey,MOVE,Smy,AEG,SUCCEEDS,Attack strength is greater,1902,spring,4,NaN
58,Turkey,SUPPORT,Con,AEG,SUCCEEDS,Unchallenged,1902,spring,4,NaN
59,Italy,HOLD,Ven,NaN,SUCCEEDS,Unchallenged,1902,spring,4,NaN
...,...,...,...,...,...,...,...,...,...,...
118,Russia,MOVE,Ukr,War,SUCCEEDS,Attack strength is greater,1902,fall,5,NaN
119,Russia,MOVE,Sev,Rum,SUCCEEDS,Attack strength is greater,1902,fall,5,NaN
120,England,BUILD,Lvp,NaN,SUCCEEDS,NaN,1902,winter,6,F
121,Italy,BUILD,Nap,NaN,SUCCEEDS,NaN,1902,winter,6,F


In [17]:
for i, dataset in enumerate(full_data_list):
    dataset.to_csv(csv_data_path + '/' + full_datanames_list[i] + '_full.csv')